In [36]:
import requests
import json
import pandas as pd

In [37]:
ListofCities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]


In [38]:
for cities in ListofCities : 
    print(cities)

Mont Saint Michel
St Malo
Bayeux
Le Havre
Rouen
Paris
Amiens
Lille
Strasbourg
Chateau du Haut Koenigsbourg
Colmar
Eguisheim
Besancon
Dijon
Annecy
Grenoble
Lyon
Gorges du Verdon
Bormes les Mimosas
Cassis
Marseille
Aix en Provence
Avignon
Uzes
Nimes
Aigues Mortes
Saintes Maries de la mer
Collioure
Carcassonne
Ariege
Toulouse
Montauban
Biarritz
Bayonne
La Rochelle


In [39]:
ListofCitiesurl = []
for i in ListofCities : 
    d="%20".join(i.split())
    ListofCitiesurl.append(d)

In [40]:
ListofCitiesurl[0]

'Mont%20Saint%20Michel'

In [41]:
l = []
for cities in ListofCitiesurl : 
    e = requests.get(f"https://nominatim.openstreetmap.org/search?q={cities}&countrycodes=FR&limit=1&format=jsonv2").json()
    l +=e
df_cities_latlon= pd.DataFrame(l)    
df_cities_latlon.head() 

with open("/Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/I) data_extract/src/cities_API.json", "w") as outfile:
     json.dump(l, outfile)

In [42]:
df_cities_latlon.head(1)

,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,place_rank,category,type,importance,icon
0,156094680,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,"Mont Saint-Michel, Plateforme du Saut-­Gaultie...",30,tourism,attraction,0.755437,https://nominatim.openstreetmap.org/ui/mapicon...


In [43]:
#round de lat long pour les utiliser comme clé de jointure entre le df_weather et le df_cities_latlon
l=[]
h=[]
for i in range(len(df_cities_latlon['lat'])) : 
    a=round(float(df_cities_latlon['lat'][i]),2)
    o=round(float(df_cities_latlon['lon'][i]),2)
    l.append(a)
    h.append(o)
df_cities_latlon['latitude']=l
df_cities_latlon['longitude']=h

In [44]:
df_cities_latlon=df_cities_latlon.drop(['lat','lon'],axis=1)

In [45]:
df_cities_latlon
#Let's see what is inside that Dataframe, I choosed CSV for 370 rows
file_name = '/Users/fezzibasma/Desktop/Data Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/gps_cities.csv'
df_cities_latlon.to_csv(file_name)
print('File is written to CSV File successfully.')


File is written to CSV File successfully.


In [46]:
# RESULT = pd.DataFrame(result)[['latitude','longitude']].reset_index().drop(['index'], axis=1)
# RESULT

In [47]:
# RIZ= pd.DataFrame(result['daily']).reset_index().drop(['index'], axis=1)
# RIZ

In [48]:
# toto=pd.concat([RIZ,RESULT],join='outer',axis=1)
# toto['latitude'] = toto['latitude'].fillna(RESULT['latitude'][0])
# toto['longitude'] = toto['longitude'].fillna(RESULT['longitude'][0])
# toto

In [49]:
# base_url variable to store url
base_url = "https://api.open-meteo.com/v1/forecast?latitude={}&longitude={}&daily=temperature_2m_max,rain_sum&windspeed_unit=ms&timezone=Europe%2FBerlin&start_date=2022-09-29&end_date=2022-10-05"

# get method of requests module
liste_latlong = []
liste_daily = []
for val in range(len(df_cities_latlon)) :
    result = requests.get(base_url.format(df_cities_latlon.latitude[val],df_cities_latlon.longitude[val])).json()
    
    #selection
    latlong = pd.DataFrame(result)[['latitude','longitude']]
    latlong['city']=ListofCities[val]
    liste_latlong.append(latlong)
    df_latlog=pd.concat(liste_latlong)
    df_latlog=df_latlog.reset_index().drop(['index'], axis=1)

    #isolation de daily 
    daily = pd.DataFrame(result['daily'])#.reset_index().drop(['index'], axis=1)
    daily['city']=ListofCities[val]
    liste_daily.append(daily)
    df_daily=pd.concat(liste_daily).reset_index().drop(['index'], axis=1)

    df_weather_ofcities=df_daily.merge(df_latlog, on='city', how='left')
    #df_weather_ofcities['latitude'] = df_weather_ofcities['latitude'].fillna(df_latlog['latitude'][val])
    #df_weather_ofcities['longitude'] = df_weather_ofcities['longitude'].fillna(df_latlog['longitude'][val])
    df_weather_ofcities

#df_weather = pd.concat([df_daily,df_latlog],axis=1,join='outer')
#df_weather

In [50]:
df_daily

,time,temperature_2m_max,rain_sum,city
0,2022-09-29,15.1,0.0,Mont Saint Michel
1,2022-09-30,16.0,0.7,Mont Saint Michel
2,2022-10-01,19.7,0.8,Mont Saint Michel
3,2022-10-02,20.0,0.0,Mont Saint Michel
4,2022-10-03,20.3,0.0,Mont Saint Michel
...,...,...,...,...
240,2022-10-01,19.0,4.2,La Rochelle
241,2022-10-02,19.2,0.0,La Rochelle
242,2022-10-03,21.7,0.0,La Rochelle
243,2022-10-04,20.7,0.0,La Rochelle


In [51]:
df_latlog

,latitude,longitude,city
0,48.6200,-1.500,Mont Saint Michel
1,48.6200,-1.500,Mont Saint Michel
2,48.6200,-1.500,Mont Saint Michel
3,48.6250,-2.000,St Malo
4,48.6250,-2.000,St Malo
...,...,...,...
100,43.5000,-1.500,Bayonne
101,43.5000,-1.500,Bayonne
102,46.1875,-1.125,La Rochelle
103,46.1875,-1.125,La Rochelle


In [53]:
#Let's see what is inside that Dataframe, I choosed CSV for 370 rows

file_name = '/Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/meteo_ofcities_7days.csv'
df_weather_ofcities.to_csv(file_name)
print('File is written to CSV File successfully.')

File is written to CSV File successfully.
